<a href="https://colab.research.google.com/github/DotBion/improved-octo-sniffle/blob/main/task_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

In [4]:
df = pd.read_csv("/content/user_transactions.csv", parse_dates=['date'], infer_datetime_format=True)

<ipython-input-4-a676fb7f898a>:1: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv("/content/user_transactions.csv", parse_dates=['date'], infer_datetime_format=True)


In [5]:
df.primaryPFC.unique()

array(['FOOD_AND_DRINK', 'GENERAL_MERCHANDISE', 'TRANSPORTATION',
       'RENT_AND_UTILITIES', 'LOAN_PAYMENTS', 'BANK_FEES',
       'PERSONAL_CARE', 'ENTERTAINMENT', 'TRAVEL', 'GENERAL_SERVICES',
       'MEDICAL', 'HOME_IMPROVEMENT', 'OTHER',
       'GOVERNMENT_AND_NON_PROFIT'], dtype=object)

**Rationale**
- I flag BNPL by finding three or more identical payments spaced about 30 ± 5 days apart—since BNPL plans split one purchase into equal monthly installments.
- I only look at transactions labeled under LOAN_PAYMENTS, then drop mortgages, car and student loans so I don’t mistake longer-term loans for BNPL.
- This also knocks out subscriptions (e.g. Netflix) that happen every month but aren’t credit-based purchases.

In [23]:
bnpl_idxs = set()
min_txns = 3
tol_low, tol_high = 25, 35

# Group by user and amount only for performance
for (user, amt), grp in df.groupby(['userId', 'amount']):
    if len(grp) < min_txns:
        continue
    grp_sorted = grp.sort_values('date')
    diffs = grp_sorted['date'].diff().dt.days.dropna()
    if diffs.between(tol_low, tol_high).all():
        bnpl_idxs.update(grp_sorted.index)

df['is_bnpl'] = df.index.isin(bnpl_idxs)

In [24]:
df_bnpl = df[(df['is_bnpl'] == True)
  & (df['primaryPFC'] == 'LOAN_PAYMENTS')
  & ~df['detailedPFC'].isin([
    'LOAN_PAYMENTS_MORTGAGE_PAYMENT',
    'LOAN_PAYMENTS_STUDENT_LOAN_PAYMENT',
    'LOAN_PAYMENTS_CAR_PAYMENT'
])]

In [26]:
df_bnpl.transactionName.unique()

array(['Withdrawal AFFIRM INC TYPE: AF', 'Withdrawal Transfer To Loan 00',
       'Debit Card Withdrawal ROCKETLA', 'Home Banking Advance To Loan 9',
       'Home Banking Withdrawal To Loa', 'Withdrawal To Loan 30',
       'Withdrawal Transfer To Loan 06', 'CAPITAL ONE',
       'CAPITAL ONE MOBILE PYMT AuthDa', 'ACH Withdrawal GreenSky TYPE: ',
       'Withdrawal Transfer To Loan 01', 'ONLINE PAYMENT',
       'ONLINE PAYMENT DEERFIELD IL pa', 'ACH Withdrawal SoFi Bank PL TY',
       'AFFIRM INC AFFIRM PAY 7522984 ', 'AFFIRM INC AFFIRM PAY 8890745 ',
       'AFFIRM INC AFFIRM PAY 9940165 ', 'SEZZLE INC sezzle.com MN 01/09',
       'SEZZLE INC sezzle.com MN 02/09', 'SEZZLE INC sezzle.com MN 12/09',
       'SYNCHRONY BANK PAYMENT 6019191', 'CAPITAL ONE AUTOPAY PYMT AuthD',
       'External Withdrawal AFFIRM INC', 'Payment to Affirm.com',
       'Point Of Sale Withdrawal AFFIR', 'Credit Card Advance IPHONE CIT',
       'Withdrawal Transfer To KIRKSEY', 'Withdrawal To Loan 90',
       'With

a. % of users who are using BNPL

In [8]:
total_users = df['userId'].nunique()
bnpl_users = df_bnpl['userId'].nunique()
pct_bnpl_users = bnpl_users / total_users * 100

In [9]:
pct_bnpl_users

14.4349477682811

b. average number of transactions in this category per month per user

In [10]:
df_bnpl['year_month'] = df_bnpl['date'].dt.to_period('M')

<ipython-input-10-20b147838d69>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bnpl['year_month'] = df_bnpl['date'].dt.to_period('M')


In [11]:
monthly_txn_counts = (
    df_bnpl
    .groupby(['userId', 'year_month'])
    .size()
    .reset_index(name='txns')
)
avg_txns_per_user_month = monthly_txn_counts['txns'].mean()

In [12]:
avg_txns_per_user_month = monthly_txn_counts['txns'].mean()


In [28]:
print(f"Average BNPL transactions per month per user : {avg_txns_per_user_month:.2f}")

Average BNPL transactions per month per user : 1.67


c. average total monthly debt burden caused by BNPL & changes YoY

In [13]:
monthly_debt = (
    df_bnpl
    .groupby(['userId', 'year_month'])['amount']
    .sum()
    .reset_index(name='debt')
)
avg_debt_per_user_month = monthly_debt['debt'].mean()


In [14]:
monthly_debt['year'] = monthly_debt['year_month'].dt.year
yearly_avg_debt = monthly_debt.groupby('year')['debt'].mean()
yoy_change = yearly_avg_debt.pct_change().dropna() * 100

In [29]:
print(f"Percent of users using BNPL: {pct_bnpl_users:.2f}%")
print(f"Average BNPL transactions per per month per user : {avg_txns_per_user_month:.2f}")
print(f"Average monthly BNPL debt per user: ${avg_debt_per_user_month:.2f}\n")

print("Yearly average BNPL debt per user:")
print(yearly_avg_debt, "\n")

print("YOY change in average BNPL debt per user (%):")
print(yoy_change)

Percent of users using BNPL: 14.43%
Average BNPL transactions per per month per user : 1.67
Average monthly BNPL debt per user: $361.84

Yearly average BNPL debt per user:
year
2024    364.809734
2025    359.981439
Name: debt, dtype: float64 

YOY change in average BNPL debt per user (%):
year
2025   -1.323511
Name: debt, dtype: float64
